In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import timedelta

In [2]:
df = pd.read_csv('../../../data/데이터정리/전체_권역별_데이터.csv')
df['날짜'] = df['날짜'].map(lambda x : datetime.datetime.strptime(x, '%Y-%m-%d').date())
df.head()

,날짜,년,월,일,시,권역,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,도심권,1.233333,0.0,False,0.0,0.333333,31.642987,993.871083
1,2017-01-01,2017,1,1,0,동남권,1.350000,0.0,False,0.0,0.500000,35.037179,1352.685069
2,2017-01-01,2017,1,1,0,동북권,0.950000,0.0,False,0.0,0.375000,30.777178,1341.727778
3,2017-01-01,2017,1,1,0,서남권,1.300000,0.0,False,0.0,0.285714,34.125789,977.771062
4,2017-01-01,2017,1,1,0,서북권,0.833333,0.0,False,0.0,0.000000,31.521646,1520.666667


In [3]:
df_서울시 = df.groupby(['날짜','년','월','일','시']).mean().reset_index()

In [4]:
df_도심권 = df[df.권역 == '도심권'].reset_index(drop=True)
df_동남권 = df[df.권역 == '동남권'].reset_index(drop=True)
df_동북권 = df[df.권역 == '동북권'].reset_index(drop=True)
df_서남권 = df[df.권역 == '서남권'].reset_index(drop=True)
df_서북권 = df[df.권역 == '서북권'].reset_index(drop=True)

## 서울시_전체

In [5]:
df_서울시

,날짜,년,월,일,시,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,1.133333,0.0,False,0.0,0.298810,32.620956,1237.344332
1,2017-01-01,2017,1,1,1,1.013333,0.0,False,0.0,0.141667,30.924628,1299.789041
2,2017-01-01,2017,1,1,2,0.522619,0.0,False,0.0,0.403571,31.022250,835.121535
3,2017-01-01,2017,1,1,3,0.260476,0.0,False,0.0,0.258333,33.496471,624.953793
4,2017-01-01,2017,1,1,4,0.048214,0.0,False,0.0,0.252381,35.248341,560.852401
...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2018-12-31,2018,12,31,19,-2.925000,0.0,False,0.0,0.429762,20.510755,2113.333029
17516,2018-12-31,2018,12,31,20,-3.521786,0.0,False,0.0,0.436905,22.331300,1948.086982
17517,2018-12-31,2018,12,31,21,-4.097500,0.0,False,0.0,0.245238,25.365712,1853.503916
17518,2018-12-31,2018,12,31,22,-4.474762,0.0,False,0.0,0.173810,27.091498,1802.200252


In [6]:
select_df = df_서울시

In [7]:
snow_day_list = select_df[select_df['강설량_체크']==True].날짜.unique()

zero_data = np.zeros((360,2))
zero_df = pd.DataFrame(zero_data, columns=['기온(°C)','계산된적설량(cm)'])

for i in snow_day_list:
    df = select_df[(select_df['날짜']<=i + timedelta(days=7))& (i - timedelta(days=7) <= select_df['날짜'])][['기온(°C)','계산된적설량(cm)']].reset_index(drop=True)
    zero_df = zero_df.add(df, fill_value=0)
    
zero_df = zero_df/len(snow_day_list)

day_list = []
time_list = []
day_time_list = []
day = -8

for d in range(15):
    day = day+1
    for t in range(24):
        day_list.append(day)
        time_list.append(t)
        day_time_list.append(str(day)+'일_'+str(t)+'시')

zero_df['day'] = day_list
zero_df['time'] = time_list
zero_df['day_time'] = day_time_list

group_zero_df = zero_df.groupby(['time']).mean().reset_index()[['time','기온(°C)','계산된적설량(cm)']]

In [8]:
zero_df

,기온(°C),계산된적설량(cm),day,time,day_time
0,-1.350389,0.019710,-7,0,-7일_0시
1,-1.655593,0.008097,-7,1,-7일_1시
2,-1.950584,0.001101,-7,2,-7일_2시
3,-2.226060,0.000000,-7,3,-7일_3시
4,-2.477803,0.000000,-7,4,-7일_4시
...,...,...,...,...,...
355,0.669137,0.012017,7,19,7일_19시
356,0.057365,0.012758,7,20,7일_20시
357,-0.394800,0.005323,7,21,7일_21시
358,-0.806819,0.004492,7,22,7일_22시


In [9]:
group_zero_df

,time,기온(°C),계산된적설량(cm)
0,0,-1.502011,0.010001
1,1,-1.791297,0.015205
2,2,-2.078178,0.006821
3,3,-2.336261,0.001751
4,4,-2.619154,0.003914
5,5,-2.897071,0.008097
6,6,-3.109764,0.009902
7,7,-3.300069,0.011324
8,8,-3.238158,0.008869
9,9,-2.318310,0.010209


In [10]:
zero_df.to_csv('../result/강설일_기준_강설량_시간별/서울시_전체.csv', index=False)

## 도심권

In [11]:
df_도심권

,날짜,년,월,일,시,권역,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,도심권,1.233333,0.0,False,0.0,0.333333,31.642987,993.871083
1,2017-01-01,2017,1,1,1,도심권,1.433333,0.0,False,0.0,0.333333,28.188683,981.193732
2,2017-01-01,2017,1,1,2,도심권,1.033333,0.0,False,0.0,0.000000,28.124563,611.801282
3,2017-01-01,2017,1,1,3,도심권,1.033333,0.0,False,0.0,0.666667,31.816060,471.964387
4,2017-01-01,2017,1,1,4,도심권,1.333333,0.0,False,0.0,0.333333,33.927185,384.500712
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2018-12-31,2018,12,31,19,도심권,-3.300000,0.0,False,0.0,0.000000,19.504344,1390.187590
17516,2018-12-31,2018,12,31,20,도심권,-3.500000,0.0,False,0.0,1.000000,21.154092,1346.766538
17517,2018-12-31,2018,12,31,21,도심권,-3.750000,0.0,False,0.0,0.333333,23.928593,1334.515076
17518,2018-12-31,2018,12,31,22,도심권,-3.800000,0.0,False,0.0,0.333333,25.663805,1312.287993


In [12]:
select_df = df_도심권

In [13]:
snow_day_list = select_df[select_df['강설량_체크']==True].날짜.unique()

zero_data = np.zeros((360,2))
zero_df = pd.DataFrame(zero_data, columns=['기온(°C)','계산된적설량(cm)'])

for i in snow_day_list:
    df = select_df[(select_df['날짜']<=i + timedelta(days=7))& (i - timedelta(days=7) <= select_df['날짜'])][['기온(°C)','계산된적설량(cm)']].reset_index(drop=True)
    zero_df = zero_df.add(df, fill_value=0)
    
zero_df = zero_df/len(snow_day_list)

day_list = []
time_list = []
day_time_list = []
day = -8

for d in range(15):
    day = day+1
    for t in range(24):
        day_list.append(day)
        time_list.append(t)
        day_time_list.append(str(day)+'일_'+str(t)+'시')

zero_df['day'] = day_list
zero_df['time'] = time_list
zero_df['day_time'] = day_time_list

group_zero_df = zero_df.groupby(['time']).mean().reset_index()[['time','기온(°C)','계산된적설량(cm)']]

In [14]:
zero_df

,기온(°C),계산된적설량(cm),day,time,day_time
0,-1.867296,0.028302,-7,0,-7일_0시
1,-2.152830,0.006289,-7,1,-7일_1시
2,-2.382075,0.000000,-7,2,-7일_2시
3,-2.599686,0.000000,-7,3,-7일_3시
4,-2.815094,0.000000,-7,4,-7일_4시
...,...,...,...,...,...
355,-0.116038,0.012579,7,19,7일_19시
356,-0.607547,0.009434,7,20,7일_20시
357,-0.984906,0.003145,7,21,7일_21시
358,-1.245912,0.003145,7,22,7일_22시


In [15]:
group_zero_df

,time,기온(°C),계산된적설량(cm)
0,0,-1.931677,0.013417
1,1,-2.191950,0.011740
2,2,-2.434235,0.005031
3,3,-2.656352,0.003354
4,4,-2.918994,0.005031
5,5,-3.182746,0.007128
6,6,-3.398218,0.009015
7,7,-3.616604,0.011950
8,8,-3.619979,0.010063
9,9,-2.939308,0.009224


In [16]:
zero_df.to_csv('../result/강설일_기준_강설량_시간별/도심권.csv', index=False)

## 동남권

In [17]:
df_동남권

,날짜,년,월,일,시,권역,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,동남권,1.350,0.0,False,0.0,0.50,35.037179,1352.685069
1,2017-01-01,2017,1,1,1,동남권,0.900,0.0,False,0.0,0.25,33.766722,1427.560417
2,2017-01-01,2017,1,1,2,동남권,0.475,0.0,False,0.0,1.75,34.232624,893.531944
3,2017-01-01,2017,1,1,3,동남권,0.000,0.0,False,0.0,0.50,36.285573,668.963889
4,2017-01-01,2017,1,1,4,동남권,-0.300,0.0,False,0.0,0.50,37.842492,604.562847
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2018-12-31,2018,12,31,19,동남권,-1.825,0.0,False,0.0,0.25,21.189535,2447.795089
17516,2018-12-31,2018,12,31,20,동남권,-2.100,0.0,False,0.0,0.25,23.568285,2153.216964
17517,2018-12-31,2018,12,31,21,동남권,-2.625,0.0,False,0.0,0.25,27.485757,2035.568899
17518,2018-12-31,2018,12,31,22,동남권,-3.200,0.0,False,0.0,0.00,29.249790,1919.265179


In [18]:
select_df = df_동남권

In [19]:
snow_day_list = select_df[select_df['강설량_체크']==True].날짜.unique()

zero_data = np.zeros((360,2))
zero_df = pd.DataFrame(zero_data, columns=['기온(°C)','계산된적설량(cm)'])

for i in snow_day_list:
    df = select_df[(select_df['날짜']<=i + timedelta(days=7))& (i - timedelta(days=7) <= select_df['날짜'])][['기온(°C)','계산된적설량(cm)']].reset_index(drop=True)
    zero_df = zero_df.add(df, fill_value=0)
    
zero_df = zero_df/len(snow_day_list)

day_list = []
time_list = []
day_time_list = []
day = -8

for d in range(15):
    day = day+1
    for t in range(24):
        day_list.append(day)
        time_list.append(t)
        day_time_list.append(str(day)+'일_'+str(t)+'시')

zero_df['day'] = day_list
zero_df['time'] = time_list
zero_df['day_time'] = day_time_list

group_zero_df = zero_df.groupby(['time']).mean().reset_index()[['time','기온(°C)','계산된적설량(cm)']]

In [20]:
zero_df

,기온(°C),계산된적설량(cm),day,time,day_time
0,-0.502830,0.009434,-7,0,-7일_0시
1,-0.866981,0.021226,-7,1,-7일_1시
2,-1.177830,0.000000,-7,2,-7일_2시
3,-1.485849,0.000000,-7,3,-7일_3시
4,-1.761792,0.000000,-7,4,-7일_4시
...,...,...,...,...,...
355,1.814623,0.016509,7,19,7일_19시
356,1.258019,0.016509,7,20,7일_20시
357,0.814151,0.007075,7,21,7일_21시
358,0.288679,0.004717,7,22,7일_22시


In [21]:
group_zero_df

,time,기온(°C),계산된적설량(cm)
0,0,-0.636289,0.005189
1,1,-0.987484,0.022956
2,2,-1.314717,0.009434
3,3,-1.610535,0.001258
4,4,-1.924780,0.005346
5,5,-2.211352,0.005975
6,6,-2.389811,0.010063
7,7,-2.546069,0.012736
8,8,-2.504717,0.008176
9,9,-1.759717,0.008176


In [22]:
zero_df.to_csv('../result/강설일_기준_강설량_시간별/동남권.csv', index=False)

## 동북권

In [23]:
df_동북권

,날짜,년,월,일,시,권역,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,동북권,0.9500,0.0,False,0.0,0.375,30.777178,1341.727778
1,2017-01-01,2017,1,1,1,동북권,0.8000,0.0,False,0.0,0.125,29.786502,1395.905556
2,2017-01-01,2017,1,1,2,동북권,0.3000,0.0,False,0.0,0.125,29.928623,928.269444
3,2017-01-01,2017,1,1,3,동북권,-0.1500,0.0,False,0.0,0.125,31.971969,707.572222
4,2017-01-01,2017,1,1,4,동북권,-0.3875,0.0,False,0.0,0.000,33.457189,639.183333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2018-12-31,2018,12,31,19,동북권,-2.7000,0.0,False,0.0,0.375,20.552894,2181.296429
17516,2018-12-31,2018,12,31,20,동북권,-3.3375,0.0,False,0.0,0.125,22.051807,2059.826190
17517,2018-12-31,2018,12,31,21,동북권,-3.7125,0.0,False,0.0,0.500,24.602845,1977.595238
17518,2018-12-31,2018,12,31,22,동북권,-4.4500,0.0,False,0.0,0.250,25.998354,1960.425000


In [24]:
select_df = df_동북권

In [25]:
snow_day_list = select_df[select_df['강설량_체크']==True].날짜.unique()

zero_data = np.zeros((360,2))
zero_df = pd.DataFrame(zero_data, columns=['기온(°C)','계산된적설량(cm)'])

for i in snow_day_list:
    df = select_df[(select_df['날짜']<=i + timedelta(days=7))& (i - timedelta(days=7) <= select_df['날짜'])][['기온(°C)','계산된적설량(cm)']].reset_index(drop=True)
    zero_df = zero_df.add(df, fill_value=0)
    
zero_df = zero_df/len(snow_day_list)

day_list = []
time_list = []
day_time_list = []
day = -8

for d in range(15):
    day = day+1
    for t in range(24):
        day_list.append(day)
        time_list.append(t)
        day_time_list.append(str(day)+'일_'+str(t)+'시')

zero_df['day'] = day_list
zero_df['time'] = time_list
zero_df['day_time'] = day_time_list

group_zero_df = zero_df.groupby(['time']).mean().reset_index()[['time','기온(°C)','계산된적설량(cm)']]

In [26]:
zero_df

,기온(°C),계산된적설량(cm),day,time,day_time
0,-1.020047,0.010950,-7,0,-7일_0시
1,-1.309906,0.003538,-7,1,-7일_1시
2,-1.627830,0.002358,-7,2,-7일_2시
3,-1.926887,0.000000,-7,3,-7일_3시
4,-2.215094,0.000000,-7,4,-7일_4시
...,...,...,...,...,...
355,1.258019,0.009434,7,19,7일_19시
356,0.629481,0.011792,7,20,7일_20시
357,0.131840,0.004717,7,21,7일_21시
358,-0.329245,0.004717,7,22,7일_22시


In [27]:
group_zero_df

,time,기온(°C),계산된적설량(cm)
0,0,-1.158237,0.007738
1,1,-1.466177,0.014465
2,2,-1.778037,0.004245
3,3,-2.052859,0.001415
4,4,-2.365196,0.002752
5,5,-2.644490,0.007075
6,6,-2.899328,0.008569
7,7,-3.123760,0.008333
8,8,-3.078037,0.008648
9,9,-2.086013,0.007469


In [28]:
zero_df.to_csv('../result/강설일_기준_강설량_시간별/동북권.csv', index=False)

## 서남권

In [29]:
df_서남권

,날짜,년,월,일,시,권역,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,서남권,1.300000,0.0,False,0.0,0.285714,34.125789,977.771062
1,2017-01-01,2017,1,1,1,서남권,1.300000,0.0,False,0.0,0.000000,33.012988,1059.479945
2,2017-01-01,2017,1,1,2,서남권,0.871429,0.0,False,0.0,0.142857,32.961913,703.324451
3,2017-01-01,2017,1,1,3,서남권,0.585714,0.0,False,0.0,0.000000,35.068174,503.712912
4,2017-01-01,2017,1,1,4,서남권,0.128571,0.0,False,0.0,0.428571,36.943293,458.848443
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2018-12-31,2018,12,31,19,서남권,-2.500000,0.0,False,0.0,0.857143,20.871024,2036.255087
17516,2018-12-31,2018,12,31,20,서남권,-3.271429,0.0,False,0.0,0.142857,22.979042,1851.434740
17517,2018-12-31,2018,12,31,21,서남권,-4.200000,0.0,False,0.0,0.142857,26.524591,1712.012987
17518,2018-12-31,2018,12,31,22,서남권,-4.457143,0.0,False,0.0,0.285714,28.433131,1670.445707


In [30]:
select_df = df_서남권

In [31]:
snow_day_list = select_df[select_df['강설량_체크']==True].날짜.unique()

zero_data = np.zeros((360,2))
zero_df = pd.DataFrame(zero_data, columns=['기온(°C)','계산된적설량(cm)'])

for i in snow_day_list:
    df = select_df[(select_df['날짜']<=i + timedelta(days=7))& (i - timedelta(days=7) <= select_df['날짜'])][['기온(°C)','계산된적설량(cm)']].reset_index(drop=True)
    zero_df = zero_df.add(df, fill_value=0)
    
zero_df = zero_df/len(snow_day_list)

day_list = []
time_list = []
day_time_list = []
day = -8

for d in range(15):
    day = day+1
    for t in range(24):
        day_list.append(day)
        time_list.append(t)
        day_time_list.append(str(day)+'일_'+str(t)+'시')

zero_df['day'] = day_list
zero_df['time'] = time_list
zero_df['day_time'] = day_time_list

group_zero_df = zero_df.groupby(['time']).mean().reset_index()[['time','기온(°C)','계산된적설량(cm)']]

In [32]:
zero_df

,기온(°C),계산된적설량(cm),day,time,day_time
0,-1.114286,0.021563,-7,0,-7일_0시
1,-1.380323,0.009434,-7,1,-7일_1시
2,-1.684367,0.000000,-7,2,-7일_2시
3,-1.936119,0.000000,-7,3,-7일_3시
4,-2.180054,0.000000,-7,4,-7일_4시
...,...,...,...,...,...
355,0.718329,0.012129,7,19,7일_19시
356,0.074169,0.013477,7,20,7일_20시
357,-0.387287,0.005391,7,21,7일_21시
358,-0.767116,0.006739,7,22,7일_22시


In [33]:
group_zero_df

,time,기온(°C),계산된적설량(cm)
0,0,-1.274468,0.012129
1,1,-1.548143,0.013657
2,2,-1.834462,0.008266
3,3,-2.079587,0.002516
4,4,-2.349551,0.002246
5,5,-2.618239,0.011500
6,6,-2.819401,0.012848
7,7,-3.001785,0.010602
8,8,-2.893196,0.009075
9,9,-1.911330,0.009614


In [34]:
zero_df.to_csv('../result/강설일_기준_강설량_시간별/서남권.csv', index=False)

## 서북권

In [35]:
df_서북권

,날짜,년,월,일,시,권역,기온(°C),강수량(mm),강설량_체크,계산된적설량(cm),교통사고수,차량통행속도,교통량
0,2017-01-01,2017,1,1,0,서북권,0.833333,0.0,False,0.0,0.000000,31.521646,1520.666667
1,2017-01-01,2017,1,1,1,서북권,0.633333,0.0,False,0.0,0.000000,29.868246,1634.805556
2,2017-01-01,2017,1,1,2,서북권,-0.066667,0.0,False,0.0,0.000000,29.863527,1038.680556
3,2017-01-01,2017,1,1,3,서북권,-0.166667,0.0,False,0.0,0.000000,32.340580,772.555556
4,2017-01-01,2017,1,1,4,서북권,-0.533333,0.0,False,0.0,0.000000,34.071547,717.166667
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17515,2018-12-31,2018,12,31,19,서북권,-4.300000,0.0,False,0.0,0.666667,20.435976,2511.130952
17516,2018-12-31,2018,12,31,20,서북권,-5.400000,0.0,False,0.0,0.666667,21.903272,2329.190476
17517,2018-12-31,2018,12,31,21,서북권,-6.200000,0.0,False,0.0,0.000000,24.286774,2207.827381
17518,2018-12-31,2018,12,31,22,서북권,-6.466667,0.0,False,0.0,0.000000,26.112413,2148.577381


In [36]:
select_df = df_서북권

In [37]:
snow_day_list = select_df[select_df['강설량_체크']==True].날짜.unique()

zero_data = np.zeros((360,2))
zero_df = pd.DataFrame(zero_data, columns=['기온(°C)','계산된적설량(cm)'])

for i in snow_day_list:
    df = select_df[(select_df['날짜']<=i + timedelta(days=7))& (i - timedelta(days=7) <= select_df['날짜'])][['기온(°C)','계산된적설량(cm)']].reset_index(drop=True)
    zero_df = zero_df.add(df, fill_value=0)
    
zero_df = zero_df/len(snow_day_list)

day_list = []
time_list = []
day_time_list = []
day = -8

for d in range(15):
    day = day+1
    for t in range(24):
        day_list.append(day)
        time_list.append(t)
        day_time_list.append(str(day)+'일_'+str(t)+'시')

zero_df['day'] = day_list
zero_df['time'] = time_list
zero_df['day_time'] = day_time_list

group_zero_df = zero_df.groupby(['time']).mean().reset_index()[['time','기온(°C)','계산된적설량(cm)']]

In [38]:
zero_df

,기온(°C),계산된적설량(cm),day,time,day_time
0,-2.247484,0.028302,-7,0,-7일_0시
1,-2.567925,0.000000,-7,1,-7일_1시
2,-2.880818,0.003145,-7,2,-7일_2시
3,-3.181761,0.000000,-7,3,-7일_3시
4,-3.416981,0.000000,-7,4,-7일_4시
...,...,...,...,...,...
355,-0.329245,0.009434,7,19,7일_19시
356,-1.067296,0.012579,7,20,7일_20시
357,-1.547799,0.006289,7,21,7일_21시
358,-1.980503,0.003145,7,22,7일_22시


In [39]:
group_zero_df

,time,기온(°C),계산된적설량(cm)
0,0,-2.517484,0.011530
1,1,-2.771006,0.013208
2,2,-3.038302,0.007128
3,3,-3.290314,0.000210
4,4,-3.544780,0.004193
5,5,-3.834738,0.008805
6,6,-4.047715,0.009015
7,7,-4.217400,0.012998
8,8,-4.100063,0.008386
9,9,-2.903061,0.016562


In [40]:
zero_df.to_csv('../result/강설일_기준_강설량_시간별/서북권.csv', index=False)